# Automatic Differentiation with AutoGrad

When training neural networks, the most frequently used algorithm is **back propagation**. In this algorithm, parameters (model weights) are adjusted according to the **gradient** of the loss function with respect to the given parameter.

To compute those gradients, PyTorch has a built-in mechanism called **AutoGrad**. It supports automatic computation of gradient for any computational graph.

### Tensors, Functions and Computational graph

The main function of PyTorch is to perform fast computations on [tensors](tensors.ipynb). Consider the following piece of code:

In [8]:
import torch
x = torch.ones(2,2,requires_grad=True)
y = x+2
print(x,y,sep='\n')

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In this code, we first define a 2x2 tensor `x`, and then compute tensor `y` based on the value of `x`. This defines the following **computational graph**: